
“Agentic” usually implies more than “good at tools”:

* **Goal-directed behavior over multiple steps**: it can decompose a task, execute steps, and keep going until a stopping condition is met (not just one tool call).
* **Planning + state/memory**: it tracks intermediate results, constraints, and what’s already been tried (often with an external state store, not just the LM).
* **Self-monitoring / control loop**: it can evaluate outcomes, detect failures, retry with a different approach, and decide when to ask for help.
* **Autonomy level**: it can initiate actions based on goals and context (within boundaries), rather than only responding.


# Some Prompts you can play around with?

Now on, I’m giving you these are some functions add(a, b) sub(a, b), mul(a, b) and div(a, b). 

NOTE: if you've to add more than 2 numbers, just call functions with more than 2 arguments, e.g. if you've 4 numbers, "1, 2, 3, 4" then you can use add(1, 2, 3, 4), likewise for multiplication, etc.

If you ever need to compute any computation involging arithmetic,  just return expresssion made from these functions, don’t try to compute numbers yourself.

Shivam has 3 apples, raji has 2 oranges, 2 apples, and shikhar has 10 bananas.   What’s total no of fruits they all have?

# Basics of Python

In [64]:
def add_something(a, b):
    print(f"Adding {a} and {b} together...")
    return a + b


add_something(**{"a": 7, "b": 12})

Adding 7 and 12 together...


19

Did you know? In Python, `**` in a function call means **“take this mapping (usually a dict) and expand it into keyword arguments.”**

### What’s happening in your call

```python
add_something(**{"a": 7, "b": 12})
```

Python sees a dict with keys `"a"` and `"b"`, and expands it as if you had written:

```python
add_something(a=7, b=12)
```

So it’s not passing the dict as a single argument. It’s **unpacking** it into named parameters.

### Why it works

Because your function is defined with parameter names `a` and `b`:

```python
def add_something(a, b):
```

…and the dict has matching keys:

```python
{"a": 7, "b": 12}
```

So `a` gets `7`, `b` gets `12`.

### Rules (the important bits)

1. **Keys must be strings** (valid keyword names, effectively):

   ```python
   add_something(**{1: 7, 2: 12})  # TypeError
   ```

2. **Keys must match parameter names**, unless the function accepts arbitrary keywords:

   ```python
   add_something(**{"x": 7, "b": 12})
   # TypeError: got an unexpected keyword argument 'x'
   ```

3. You can mix normal args + `**` (but don’t give the same value twice):

   ```python
   add_something(7, **{"b": 12})      # ok -> a=7, b=12
   add_something(a=7, **{"a": 9})     # TypeError: multiple values for 'a'
   ```

4. You can also merge multiple `**` expansions:

   ```python
   add_something(**{"a": 7}, **{"b": 12})
   ```

### Related: `*` vs `**`

* `*` unpacks a sequence into **positional arguments**:

  ```python
  add_something(*[7, 12])   # same as add_something(7, 12)
  ```

* `**` unpacks a mapping into **keyword arguments**:

  ```python
  add_something(**{"a": 7, "b": 12})  # same as add_something(a=7, b=12)
  ```

If you want, I can show how this interacts with `def f(*args, **kwargs)` which is where `**` really becomes obvious.


---
## 1 · Why Do We Need Function Calling?

LLMs are impressive — they can write essays, explain quantum physics, draft emails. But they have some hard limits:

- 🧊 **Frozen in time** — they don't know what happened after their training cutoff
- 🔢 **Bad at precise computation** — they *predict* math answers rather than *computing* them
- 🚫 **Can't take actions** — they can't send emails, query databases, or call APIs

Let's actually *see* these failures before we fix them.


### 1.2 Load Your API Key Securely

We **never** hardcode API keys. Instead we keep them in a `.env` file and load them at runtime with `python-dotenv`. Treat your key like a password — if it leaks, anyone can run up charges on your account.


In [113]:
import os, json
from dotenv import load_dotenv

import textwrap



def pretty_print(*args):
    text = " ".join(str(arg) for arg in args)
    try:
        print(textwrap.fill(text, width=80))
    except Exception as e:
        print(text)  # fallback to normal print if text is not a string

        

load_dotenv('/Users/shivam13juna/Documents/scaler/iitr_classes/llm_ref/openai_key.env')  # reads .env file in the current directory

api_key = os.getenv("OPENAI_API_KEY")

if not api_key:
    raise ValueError(
        "OPENAI_API_KEY not found! "
        "Make sure you have a .env file with: OPENAI_API_KEY=sk-..."
    )

pretty_print("API key loaded successfully.")

API key loaded successfully.


In [114]:
pretty_print("a" + "b")

ab


In [115]:
import truststore
truststore.inject_into_ssl()

#This is optional. I use VPN in my computer. Why I have to use this



### 1.3 Initialize the OpenAI Client

The `OpenAI` class is our gateway to every model endpoint. We create it once and reuse it throughout the notebook.


In [116]:
from openai import OpenAI

client = OpenAI(api_key=api_key)
pretty_print("OpenAI client ready.")

MODEL  = "gpt-5-nano"  

OpenAI client ready.


### 🧪 Experiment 1: Ask about real-time information

In [39]:
# The model has NO access to live data — what does it do?

response = client.responses.create(
    model=MODEL,
    input="What is the weather in bengaluru right now? Give me the exact temperature.",
    reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
print(response.output_text)

I don’t have real-time weather access. You can check the current temperature for Bengaluru on a weather service (e.g., Weather.com, AccuWeather, or your device’s weather app) for the exact value. If you share a location or time, I can help interpret forecast data.


☝️ The model either **admits it can't check**, or worse — **makes up a plausible-sounding answer**. Either way, it's not helpful for a user who needs real weather data.

### 🧪 Experiment 2: Ask for precise math


In [41]:
# The model PREDICTS math — it doesn't COMPUTE it

response = client.responses.create(
    model=MODEL,
    input="What is 1247 * 83 + 19 / 3.7? Give me answer in one line (max 10 words), the exact number, upto 2 decimal places.",
    reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
print("Model says:", response.output_text)

# What Python actually computes
import math
actual = 1247 * 83 + 19 / 3.7
print(f"Actual answer: {actual}")


Model says: 103,700.00
Actual answer: 103506.13513513513


Sometimes the model gets it right, sometimes it's off. The point is — it's **guessing**, not computing. For a banking app or scientific tool, "usually right" isn't good enough.

### 🧪 Experiment 3: Ask it to take an action


In [43]:
# Can the model actually DO something in the real world?

response = client.responses.create(
    model=MODEL,
    input="Send an email to john@example.com saying 'Meeting moved to 3pm'.",
    reasoning={"effort": "minimal"},   
    text={"verbosity": "low"}
)
print(response.output_text)

I can draft the email for you. Here’s a plain text version you can send:

To: john@example.com
Subject: Meeting time updated
Body:
Hi John,

Meeting moved to 3pm.

Best regards,
[Your Name]

If you’d like, tell me your name and I can tailor it or format for your email client.


The model can *write* the email — but it can't *send* it. It has no access to your email server, your contacts, or anything outside its text-generation bubble.

---

### So what do we actually want?

We want the model to be able to say:

> *"I can't compute this myself, but I know there's a calculator function available. Let me ask for `compute_math(expression='1247 * 83 + 19 / 3.7')` and use the result."*

> *"I can't check live weather, but I know there's a `get_weather` function. Let me call `get_weather(location='Tel Aviv')` and report back."*

**That's function calling.** The model doesn't gain superpowers — instead, it learns to **ask for help** from functions *you* provide.


---
## 1.1 · The Key Vocabulary

Three terms you'll see everywhere. Let's nail them down with a concrete example — getting today's weather.

### 🔧 Tool (or Function)
A piece of functionality **you** write and **describe** to the model. The model never sees your code — it only sees a name, a description, and a schema of what arguments the function accepts.

```
Tool name:        get_weather
Description:      "Get the current temperature for a city"
Expected input:   { "location": "Tel Aviv" }
```

### 📞 Tool Call (or Function Call)  
When the model decides it needs a tool, it doesn't answer with text. Instead it returns a **structured request**: *"Please call this function with these arguments."*

```
Model's response:  function_call → get_weather(location="Tel Aviv")
                   (NOT "The weather in Tel Aviv is 28°C")
```

### 📦 Tool Call Output (or Function Call Output)
**Your code** actually runs the function, gets a result, and sends it back. The model then uses that real data to compose its final answer.

```
Your code runs:    get_weather("Tel Aviv") → {"temp": 28, "unit": "C"}
You send back:     function_call_output with that result
Model finally:     "The current temperature in Tel Aviv is 28°C."
```


---
## 1.2 · See It Happen — The Model Outputs a Function Call, Not Text

Let's give the model a simple `add` function and ask it to add two numbers. Watch what comes back — it's **not** a text answer.


In [110]:
# Define a simple "add" tool — we'll explain the structure in detail next
add_tool = {
    "type": "function",
    "name": "add",
    "description": "Add two numbers together and return the sum.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}


# Ask the model to add — but give it the tool
response = client.responses.create(
    model=MODEL,
    instructions="Use the add tool for any math. Never compute math yourself.",
    input="What is 7 + 12?",
    tools=[add_tool],
)

# Let's inspect what came back
print("Output items from the model:")
print("-" * 40)
for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

Output items from the model:
----------------------------------------
  Type: reasoning
  Type: function_call
  Function name: add
  Arguments: {"a":7,"b":12}
  Call ID: call_DeRCb62JAFlxtyq8U0jbtSVx
 Response:


In [111]:
# Ask the model to add — but give it the tool
response = client.responses.create(
    model=MODEL,
    instructions="Use the add tool for any math. Never compute math yourself.",
    input="Why is Earth round?",
    tools=[add_tool],
)


# Let's inspect what came back
print("Output items from the model:")
print("-" * 40)
for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

Output items from the model:
----------------------------------------
  Type: reasoning
  Type: message
 Response: Earth isn’t perfectly round, but it is round in the sense that
gravity pulls all its mass toward the center, making the overall shape as close
to a sphere as possible.  - Why a sphere? For a non-rotating chunk of matter,
gravity tends to smooth itself into a sphere—the shape with the smallest surface
area for a given volume. - Why not a perfect sphere? Earth rotates. The spin
creates centrifugal force that pushes material outward at the equator, producing
a bulge. The result is an oblate spheroid (a sphere that’s slightly flattened at
the poles). - How big is the bulge? The equatorial radius is about 6,378 km, the
polar radius about 6,357 km—so the difference is small, but real. - What about
mountains and oceans? They make local bumps, but they don’t change the overall
rounded shape much. The true “mean” shape is described by the geoid, which is an
irregular, slightly wavy

In [ ]:
# Adding more tools doesn't change the model's behavior if the question doesn't require them

add_tool = {
    "type": "function",
    "name": "add",
    "description": "Add two numbers together and return the sum.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

sub_tool = {
    "type": "function",
    "name": "subtract",
    "description": "Subtract two numbers and return the difference.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

mul_tool = {
    "type": "function",
    "name": "multiply",
    "description": "Multiply two numbers together and return the product.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}


div_tool = {
    "type": "function",
    "name": "divide",
    "description": "Divide two numbers and return the answer.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "Numerator (dividend)"},
            "b": {"type": "number", "description": "Denominator (divisor)"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b

def divide(a, b):
    if b == 0:
        return "Error: Division by zero"
    return a / b

# Ask the model to add — but give it the tool
response = client.responses.create(
    model=MODEL,
    instructions="Use the tools for any math. Never compute math yourself.",
    input="What is 1247 * 83 + 19 / 3.7?",
    tools=[add_tool, sub_tool, mul_tool, div_tool],
    reasoning={"effort": "minimal"}
)


# Let's inspect what came back
print("Output items from the model:")
print("-" * 40)
for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)


Output items from the model:
----------------------------------------
  Type: reasoning
  Type: function_call
  Function name: multiply
  Arguments: {"a":1247,"b":83}
  Call ID: call_zDpxNvVXzscGt6IG27J1wUsH
  Type: function_call
  Function name: divide
  Arguments: {"a":19,"b":3.7}
  Call ID: call_5n1T83Td1NATlVPKXVXl6gcq
 Response:


🎯 **Look at that!** The model did NOT respond with "19". Instead it returned:

- `type: "function_call"` — it's requesting a tool call
- `name: "add"` — it picked our function
- `arguments: {"a": 7, "b": 12}` — it figured out the right arguments from the user's question

The model **proposed** a function call. It's now waiting for us to **execute** it and give back the result.

Let's try another one — a weather function:


In [57]:
# Define a get_weather tool
weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get the current temperature for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "City name, e.g. 'Tel Aviv', 'London'"
            },
        },
        "required": ["location"],
        "additionalProperties": False,
    },
    "strict": True,
}

# Same question as before — but now the model has a tool!
response = client.responses.create(
    model=MODEL,
    input="What's the weather in Paris right now?",
    tools=[weather_tool],
)

# Let's inspect what came back
print("Output items from the model:")
print("-" * 40)
for item in response.output:
    print(f"  Type: {item.type}")
    if item.type == "function_call":
        print(f"  Function name: {item.name}")
        print(f"  Arguments: {item.arguments}")
        print(f"  Call ID: {item.call_id}")
pretty_print(" Response: " + response.output_text)

Output items from the model:
----------------------------------------
  Type: reasoning
  Type: function_call
  Function name: get_weather
  Arguments: {"location":"Paris"}
  Call ID: call_lrkyH4yEt9AWVmiY7c5KTMly
 Response:


Without the tool, the model would have guessed or refused. **With** the tool, it says *"call `get_weather` with `location='Paris'`"* — and waits for real data.

This is the fundamental shift: **the model stops pretending to know things and starts asking for help.**


---
## 1.3 · Anatomy of a Tool Definition

Let's break down exactly what we just wrote. Every tool definition has these parts:

```python
{
    "type": "function",          # Always "function" for custom tools
    "name": "add",               # What the model calls it by
    "description": "Add two numbers together.",   # Helps the model decide WHEN to use it
    "parameters": {              # JSON Schema describing the arguments
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],           # Which fields are mandatory
        "additionalProperties": False,    # No extra fields allowed (needed for strict)
    },
    "strict": True,              # Guarantee arguments match schema exactly
}
```

**Think of it like a contract:**  
- The `description` tells the model *"here's what this tool does — use it when relevant"*
- The `parameters` schema tells the model *"here's exactly what arguments to generate"*
- `strict: True` tells the API *"enforce this schema — reject anything that doesn't match"*

> The model never sees your Python function code. It only sees this definition.


---
## 1.4 · The Complete Flow — End to End

Now let's put all five steps together with our simple `add` function. This is the pattern you'll use for everything.


In [75]:
import json

# Step 0: Our actual Python function (the model never sees this!)
def add(a, b):
    return a + b

# Step 1: Call the API with tools defined
response = client.responses.create(
    model=MODEL,
    instructions="Always use the add tool for math. Never compute yourself.",
    input="What is 7 + 12?",
    tools=[add_tool],       # from earlier
)

print("═══ Step 2: Model returns a function_call (not text!) ═══")
for item in response.output:
    print(f"  {item.type}: ", end="")
    if item.type == "function_call":
        print(f"{item.name}({item.arguments})")
    else:
        print(getattr(item, 'content', ''))

═══ Step 2: Model returns a function_call (not text!) ═══
  reasoning: None
  function_call: add({"a":7,"b":12})


In [76]:
# Step 3: We execute the function with the model's arguments
function_call = [item for item in response.output if item.type == "function_call"][0]
args = json.loads(function_call.arguments)

result = add(**args)       # add(a=7, b=12) → 19
print(f"═══ Step 3: We ran add({args}) → {result} ═══")

═══ Step 3: We ran add({'a': 7, 'b': 12}) → 19 ═══


In [77]:
# Step 4: Send the result back to the model
# We need to include:
#   - The model's original output (preserves its chain of thought)
#   - Our function_call_output (the actual result, linked by call_id)

#followup_input = list(response.output)     # the model's original items
followup_input = []
followup_input.append({
    "type": "function_call_output",        # this is OUR result going back
    "call_id": function_call.call_id,      # links to the specific function_call
    "output": str(result),                 # must be a string
})

# Step 5: Model writes the final answer using the real result
final = client.responses.create(
    model=MODEL,
    instructions="Always use the add tool for math. Never compute yourself.",
	previous_response_id=response.id,   # links to the previous response
    input=followup_input,
    tools=[add_tool],
)

print(f"═══ Step 5: Final answer ═══")
print(final.output_text)

═══ Step 5: Final answer ═══
19


In [71]:
followup_input

[ResponseReasoningItem(id='rs_0ef7ba8b393cd93e0069a025ac55cc819888459701fb5271fa', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseFunctionToolCall(arguments='{"a":7,"b":12}', call_id='call_NKKfTWChw4XGnOQnUm4K6unr', name='add', type='function_call', id='fc_0ef7ba8b393cd93e0069a025ae2a108198b9ec7e9d531489f5', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_NKKfTWChw4XGnOQnUm4K6unr',
  'output': '19'}]

### Let's see the same flow for `get_weather`:

In [ ]:
# Our fake weather function (in production, this would call a real API)
def get_weather(location):
    fake_data = {"Tel Aviv": "28°C, sunny", "Paris": "18°C, cloudy", "London": "14°C, rain"}
    return fake_data.get(location, f"No data for {location}")

# Step 1: Ask with the weather tool
response = client.responses.create(
    model=MODEL,
    input="What's the weather like in Tel Aviv and London?",
    tools=[weather_tool],
)

# Step 2 & 3: Find all function calls, execute each
print("Model requested these calls:")
new_input = list(response.output)

for item in response.output:
    if item.type == "function_call":
        args = json.loads(item.arguments)
        result = get_weather(**args)
        print(f"  → {item.name}({args}) = {result}")
        
        # Step 4: Append our result
        new_input.append({
            "type": "function_call_output",
            "call_id": item.call_id,
            "output": result,
        })

# Step 5: Model composes final answer with real data
final = client.responses.create(
    model=MODEL,
    input=new_input,
    tools=[weather_tool],
)
print(f"\nFinal answer:\n{final.output_text}")

### 🧠 What just happened — step by step

```
1. You  ──▶  API      "What's the weather?" + tool definitions
2.           API ──▶   Model returns function_call items (not text!)
3. You execute the function(s) with the model's arguments
4. You  ──▶  API      Send back function_call_output for each call
5.           API ──▶   Model writes a final answer using real data
```

The model **proposes**. Your app **executes**. The model **summarizes**.

That's it — that's the entire pattern. Everything from here builds on exactly this loop.


### Same flow but with more tools and more calls

In [117]:
# Adding more tools doesn't change the model's behavior if the question doesn't require them

add_tool = {
    "type": "function",
    "name": "add",
    "description": "Add two numbers together and return the sum.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

sub_tool = {
    "type": "function",
    "name": "subtract",
    "description": "Subtract two numbers and return the difference.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

mul_tool = {
    "type": "function",
    "name": "multiply",
    "description": "Multiply two numbers together and return the product.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "First number"},
            "b": {"type": "number", "description": "Second number"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}


div_tool = {
    "type": "function",
    "name": "divide",
    "description": "Divide two numbers and return the answer.",
    "parameters": {
        "type": "object",
        "properties": {
            "a": {"type": "number", "description": "Numerator (dividend)"},
            "b": {"type": "number", "description": "Denominator (divisor)"},
        },
        "required": ["a", "b"],
        "additionalProperties": False,
    },
    "strict": True,
}

def add(a, b):
    return a + b

def subtract(a, b):
    return a - b

def multiply(a, b):
    return a * b

def divide(a, b):
    if b == 0:
        return "Error: Division by zero"
    return a / b


DISPATCH = {
    "add": add,
    "subtract": subtract,
    "multiply": multiply,
    "divide": divide,
}

TOOLS = [add_tool, sub_tool, mul_tool, div_tool]





DEV_POLICY = "Use the tools for any math. Never compute math yourself."

def answer_with_math_tools_verbose(user_question: str) -> str:
    print("══════════════════════════════════════════════════════")
    print("START: User question")
    print("  ", user_question)
    print("══════════════════════════════════════════════════════")

    # Round 1: seed conversation with developer policy + user question
    resp = client.responses.create(
        model=MODEL,
        input=[
            {"role": "developer", "content": DEV_POLICY},
            {"role": "user", "content": user_question},
        ],
        tools=TOOLS
    )

    round_num = 1

    while True:
        print(f"\n══════════════════════════════════════════════════════")
        print(f"ROUND {round_num}: Model response items")
        print("══════════════════════════════════════════════════════")

        # Show every output item type
        for i, item in enumerate(resp.output):
            print(f"[{i}] type = {item.type}")

            if item.type == "function_call":
                print(f"    name     = {item.name}")
                print(f"    call_id  = {item.call_id}")
                print(f"    arguments(raw JSON string) = {item.arguments}")

            elif item.type == "message":
                # Some SDKs represent assistant text as message items
                # output_text is the easiest way to get the final combined text.
                try:
                    print(f"    content = {item.content}")
                except Exception:
                    pass

        calls = [item for item in resp.output if item.type == "function_call"]

        # If no tool calls, we’re done; return final user-facing text
        if not calls:
            print("\n✅ No function calls. Final assistant text:")
            print(resp.output_text)
            return resp.output_text

        print("\n→ Model requested tool calls:")
        for call in calls:
            print(f"  - {call.name}({call.arguments})  [call_id={call.call_id}]")

        # Execute all calls and prepare tool outputs
        tool_outputs = []
        print("\n→ Executing tools locally (your server/app):")
        for call in calls:
            fn = DISPATCH.get(call.name)
            args = json.loads(call.arguments)

            try:
                result = fn(**args)
                payload = {"ok": True, "result": result}
                print(f"  ✓ {call.name}(**{args}) -> {result}")
            except Exception as e:
                payload = {"ok": False, "error": str(e)}
                print(f"  ✗ {call.name}(**{args}) -> ERROR: {e}")

            tool_outputs.append({
                "type": "function_call_output",
                "call_id": call.call_id,
                # output is a string; keep it JSON for readability + structure
                "output": json.dumps(payload),
            })

        print("\n→ Sending tool outputs back to the model:")
        for out in tool_outputs:
            pretty_print(out)

        # Continue conversation, ONLY sending tool outputs (chained with previous_response_id)
        resp = client.responses.create(
            model=MODEL,
            previous_response_id=resp.id,
            input=tool_outputs,
            tools=TOOLS,
            reasoning={"effort": "minimal"},
        )

        round_num += 1

# Example
final_text = answer_with_math_tools_verbose("What is 1247 * 83 + 19 / 3.7?")

══════════════════════════════════════════════════════
START: User question
   What is 1247 * 83 + 19 / 3.7?
══════════════════════════════════════════════════════

══════════════════════════════════════════════════════
ROUND 1: Model response items
══════════════════════════════════════════════════════
[0] type = reasoning
[1] type = function_call
    name     = multiply
    call_id  = call_59EkQjUlhuBV4MPWbYSSNzvL
    arguments(raw JSON string) = {"a":1247,"b":83}
[2] type = function_call
    name     = divide
    call_id  = call_8MMWUDDBvJSrNwZO9IQ2dgqy
    arguments(raw JSON string) = {"a":19,"b":3.7}

→ Model requested tool calls:
  - multiply({"a":1247,"b":83})  [call_id=call_59EkQjUlhuBV4MPWbYSSNzvL]
  - divide({"a":19,"b":3.7})  [call_id=call_8MMWUDDBvJSrNwZO9IQ2dgqy]

→ Executing tools locally (your server/app):
  ✓ multiply(**{'a': 1247, 'b': 83}) -> 103501
  ✓ divide(**{'a': 19, 'b': 3.7}) -> 5.135135135135135

→ Sending tool outputs back to the model:
{'type': 'function_cal

---
## 1.5 · Quick Recap

| Without function calling | With function calling |
|---|---|
| "The weather is probably around 25°C" | Calls `get_weather("Tel Aviv")` → gets **real** 28°C |
| "1247 × 83 = 103,501" (maybe wrong) | Calls `compute_math("1247*83+19/3.7")` → gets **exact** answer |
| "I can't send emails" | Calls `send_email(to="john@...", body="...")` → email **actually sent** |

**The model's job changed:** instead of *pretending to know everything*, it now *decides which tool to call and with what arguments*. Your code does the rest.

### Responses API vs Chat Completions — what changed?

We're using the **Responses API** (`client.responses.create`) throughout this notebook — it's OpenAI's recommended API going forward.

| Chat Completions (old) | Responses API (new) |
|---|---|
| `client.chat.completions.create()` | `client.responses.create()` |
| `messages=[{role, content}]` | `input=[{role, content}]` or just a string |
| System message in `messages` | `instructions="..."` parameter |
| `response.choices[0].message.content` | `response.output_text` |
| `response.choices[0].message.tool_calls` | `response.output` items with `type=="function_call"` |
| Tool result: `{role:"tool", tool_call_id:..., content:...}` | `{type:"function_call_output", call_id:..., output:...}` |
| `response_format={type:"json_schema",...}` | `text={format:{type:"json_schema",...}}` |

Now let's build something more interesting — starting with a proper calculator tool.


# create a tool of weather API